# **Machine Learning Project**

In questa esercitazione metteremo assieme tutte le nozioni apprese dall'inizio del corso per risolvere un task specifico di machine learning.

### **Task: Semi-supervised classification**

Il task che vogliamo risolvere è un task di classificazione, caratterizzato però dal fatto che solo una piccola parte dei dati che disponiamo possiede le annotazioni (label). Questa condizione è nota come `Semi-supervised learning`. 

### **Dataset**

Il dataset che utilizzeremo sarà Fashion-MNIST, che contiene immagini di articoli di Zalando, composto da un training set di 60.000 campioni e test set con 10.000 campioni. Ogni campione è in scala di grigi e ha risoluzione 28x28. Il dataset è composto da 10 classi.


## **Pseudo-label**

Lo **pseudo-labeling** è una tecnica utilizzata nell'ambito del *semi-supervised learning*. L'idea di base è quella di generare etichette "artificiali" (pseudo-etichette) per i dati non etichettati (unlabeled, "U"), in modo da utilizzarle durante il training del modello. Per generare queste etichette ci sono diverse strategie: nel contesto di questa esercitazione utilizzeremo un algoritmo di clustering (k-means).

Ecco i passaggi generali del processo di pseudo-labeling:

1.  **Addestramento Iniziale**: Si addestra un algortimo di clustering sul set non etichettato (U) utilizzando un numero di cluster pari al numero di classi.
2.  **Predizione su Dati Etichettati**: Utilizziamo l'algortimo addestrato al punto 1 per clusterizzare i dati etichettati (L), assegnandoli quindi ai cluster che abbiamo trovato durante l' addestramento iniziale.
3.  **Mappare i cluster alle etichette**: Creiamo un mapping tra i cluster e le etichette, in modo da capire quale etichetta corrisponde allo specifico cluster. Per fare ciò assegniamo ad ogni cluster la vera label più frequente assegnata a quel cluster, sfruttando la funzione `mode`:

```Python
from scipy.stats import mode
import numpy as np

etichette_nel_cluster = np.array([0, 1, 1, 2, 1, 0, 1])

risultato_mode = mode(etichette_nel_cluster)

print(f"Oggetto ModeResult: {risultato_mode}") # Output: ModeResul(mode=1, count=4)
print(f"Etichetta più frequente (moda): {risultato_mode.mode}") # Output: (moda): 1

# etichetta più frequente come singolo numero:
etichetta_predominante = risultato_mode.mode
print(f"Etichetta predominante per questo cluster: {etichetta_predominante}") # Output: 1
```

4.  **Estrazione pseudo-label**: Alla fine, la classe più presente in un cluster diventa l' etichetta scelta per tutti i campioni assegnati a quel cluster. 


**Vantaggi**:
*   Permette di sfruttare la grande quantità di dati non etichettati, che altrimenti andrebbero sprecati.
*   Può migliorare significativamente le prestazioni del modello rispetto all'addestramento con i soli dati etichettati, specialmente quando questi ultimi sono scarsi.

In [ ]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import mode # For majority voting
from sklearn.neural_network import MLPClassifier

In [ ]:
# Nomi delle classi per Fashion-MNIST
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### `load_and_preprocess_data()`

In questa funzione dovrete:

* Scaricare il dataset.
* Riordinare casualmente i dati.
* Effettuare reshape.
* Scalare i valori dei pixel all' intervallo [0,1].
* Ridurre il numero di campioni a 10.000 per il train e 1.000 per il test.

La funzione dovrà ritornare nel seguente ordine:

1. Il training set ridotto.
2. Le etichette di train ridotte.
3. Il test set ridotto.
4. Le etichette di test ridotte.

In [ ]:
def load_and_preprocess_data():
    """Carica e pre-processa il dataset Fashion-MNIST."""
    np.random.seed(0)

    # 1. Carica i dati
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

    # 2. Riordina casualmente i dati
    indices = np.arange(x_train.shape[0])
    
    np.random.shuffle(indices)
    x_train = x_train[indices]
    y_train = y_train[indices]

    #effettuo reshape 
    x_train = x_train.astype('float32')/255.0
    x_test = x_test.astype('float32')/255.0

    x_train_reduced = x_train[:8000]
    x_test_reduced = x_test[:1000]
    y_train_reduced = y_train[:8000]
    y_test_reduced = y_test[:1000]
    # 6. Ritorna i dati nel giusto ordine
    return x_train_reduced, y_train_reduced, x_test_reduced, y_test_reduced
load_and_preprocess_data()

### `apply_pca_and_scale`

In questa funzione dovrete:

* Scalare il training set e il test set.
* Applicare PCA con un numero di componenti specificato come parametro della funzione (o, equivalentemente, con una frazione desiderata della varianza espressa).
* Stampare il numero di componenti.
* Stampare la varianza espressa.

La funzione dovrà ritornare nel seguente ordine:

1. Il training set trasformato con PCA.
2. Il test set trasformato con PCA.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def apply_pca_and_scale(x_train, x_test, n_components):

    x_train = x_train.reshape(len(x_train), -1)
    x_test = x_test.reshape(len(x_test), -1)

    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    pca = PCA(n_components=n_components, random_state=42)
    x_train_pca = pca.fit_transform(x_train_scaled)
    x_test_pca = pca.transform(x_test_scaled)

    print(f"PCA - Numero componenti: {pca.n_components_}")
    print(f"PCA - Varianza spiegata totale: {np.sum(pca.explained_variance_ratio_):.4f}")

    return x_train_pca, x_test_pca

### `create_semi_supervised_split`

In questa funzione dovrete:

* Splittare il training set in due insiemi, etichettato (L) e non etichettato (U) utilizzando  `train_test_split` con:

`test_size`=`(1.0 - labeled_fraction)`

* Stampare la shape del set etichettato.
* Stampare la shape del set non etichettato.

La funzione deve ritornare nell seguente ordine:

1. Il set etichettato.
2. Le etichette del set etichettato.
3. Il set non etichettato.
4. Le etichette del set non etichettato. **N.B.** Queste etichette verranno utilizzate **SOLO** per valutare le pseudo-labels, non per l'addestramento.


In [ ]:
def create_semi_supervised_split(x_train_pca, y_train, labeled_fraction):
    
    # Split del training set
    x_labeled, x_unlabeled, y_labeled, y_unlabeled = train_test_split(
        x_train_pca, y_train,
        test_size=(1.0 - labeled_fraction),
        stratify=y_train,
        random_state=0
    )

    # Stampa delle shape
    print(f"Set etichettato (L) shape: {x_labeled.shape}")
    print(f"Set non etichettato (U) shape: {x_unlabeled.shape}")

    return x_labeled, y_labeled, x_unlabeled, y_unlabeled

### `get_pseudo_labels`

In questa funzione dovrete:

* Istanziare un algoritmo di clustering (ad esempio, k-means).
* Addestrare e predire i clustering sul set non etichettato.
* Predire i clustering del set etichettato.
* Mappare i cluster ad un etichetta, utilizzando per ogni cluster l'etichetta più presente, estraibile utilizzando la funzione `mode` presentata sopra.
* Generare un array `pseudo_labels` assegnando a ogni campione del set non etichettato l'etichetta corrispondente al cluster a cui è stato assegnato.

La funzione deve ritornare:

1. L' array `pseudo_labels`.

In [ ]:

def get_pseudo_labels(x_unlabeled, x_labeled, y_labeled, n_clusters=10):
    """Genera pseudo-labels usando clustering + majority voting."""

    # 1. Clustering sui dati non etichettati
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(x_unlabeled)
      
    # 2. Cluster assignments
    cluster_assignments_unlabeled = kmeans.predict(x_unlabeled)
    cluster_assignments_labeled = kmeans.predict(x_labeled)

    # 3. Mappatura cluster → etichetta
    cluster_to_true_label_map = {}
    for k_idx in range(n_clusters):
        # Trova indici dei dati etichettati assegnati a questo cluster
        indices = np.where(cluster_assignments_labeled == k_idx)[0]
        if len(indices) > 0:
            labels = y_labeled[indices]
            # Trova l’etichetta più frequente (moda)
            most_common = mode(labels, keepdims=False).mode
            cluster_to_true_label_map[k_idx] = most_common
        else:
            # Nessun dato etichettato in questo cluster
            cluster_to_true_label_map[k_idx] = np.nan

    # 4. Assegna pseudo-labels
    pseudo_labels_list = []
    for c_assign in cluster_assignments_unlabeled:
        label = cluster_to_true_label_map.get(c_assign, np.nan)
        pseudo_labels_list.append(label)

    # 5. Converti in array
    pseudo_labels = np.array(pseudo_labels_list)

    return pseudo_labels


### `train_and_evaluate_classifier`

In questa funzione dovrete:

* Rimuovere eventuali campioni con etichette NaN (potrebbero provenire da pseudo labels non mappate).
* Istanziare il modello utilizzando `model_class` come oggetto e `classifier_args` come argomenti. Esempio:

```Python
model_class = MLPClassifier
classifier_args = {'max_iter': 200, 'hidden_layer_sizes': (100, 50)}
model = model_class(**classifier_args)

# Equivalente a:
model = MLPClassifier(max_iter=200, hidden_layer_sizes=(100, 50))

```

* Allenare il modello sul training set a cui sono stati rimossi i campioni con etichette NaN.
* Calcolare l' accuracy.
* Stampare il `title`, che consiste nel titolo dell' esperimento eseguito. Questo perchè tale funzione verrà riutilizzata diverse volte per più set di dati. Un titolo ci permetterà di identificare quali risultati stiamo producendo.
* Stampare l' accuracy.
* Stampare il classification report.

La funzione deve ritornare:

1. Il modello.


In [ ]:
def train_and_evaluate_classifier(model_class, classifier_args, x_train, y_train, x_test, y_test, title=None, class_names_list=None):
    valid_indices_train = ~np.isnan(y_train)
    x_train_c = x_train[valid_indices_train]
    y_train_c = y_train[valid_indices_train]

    model = model_class(**classifier_args)
    model.fit(x_train_c, y_train_c)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{title} - Accuracy: {accuracy:.4f}")

    if class_names_list is not None:
        from sklearn.metrics import classification_report
        print(classification_report(y_test, y_pred, target_names=class_names_list))

    return model


### `main`

In questa funzione dovrete:

* Utilizzare la funzione `load_and_preprocess_data` per caricare e pre-processare i dati.
* Utilizzare la funzione `apply_pca_and_scale` per applicare scaling e PCA.
* Utilizzare la funzione `create_semi_supervised_split` per dividere il train set in set etichettato e non etichettato.
* Utilizzare la funzione `get_pseudo_labels` per calcoalre le pseudo labels sul set non etichettato.
* Calcolare l' accuracy delle pseudo labels, cioè confrontarle con quelle vere in modo da vedere quanto sono accurate.
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello solo sui dati etichettati (L).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello solo sui dati non etichettati (U).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello sui dati etichettati (L) più quelli non etichettati (U).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello su tutto il dataset originale.

In [ ]:
def main(classifier_class, classifier_args, n_components_pca, labeled_fraction, n_clusters):
    x_train, y_train, x_test, y_test = load_and_preprocess_data()

    x_train_pca, x_test_pca = apply_pca_and_scale(x_train, x_test, n_components_pca)

    x_etichettato, y_etichettato, x_nonetichettato, y_nonetichettato = create_semi_supervised_split(x_train_pca, y_train, labeled_fraction)

    pseudo_labels = get_pseudo_labels(x_nonetichettato, x_etichettato, y_etichettato, n_clusters)

    mask = ~np.isnan(pseudo_labels) 
    pseudo_accuracy = accuracy_score(y_nonetichettato[mask], pseudo_labels[mask])
    print(f"\nAccuracy delle pseudo-labels: {pseudo_accuracy:.4f}\n")

    print(" Modello solo su dati etichettati :")
    train_and_evaluate_classifier(classifier_class, classifier_args, x_etichettato, y_etichettato, x_test_pca, y_test)

    print("\n Modello solo su pseudo-labels :")
    train_and_evaluate_classifier(classifier_class, classifier_args, x_nonetichettato[mask], pseudo_labels[mask], x_test_pca, y_test)

    print("\n Modello su dati etichettati e pseudo-labels :")
    x_combined = np.concatenate([x_etichettato, x_nonetichettato[mask]])
    y_combined = np.concatenate([y_etichettato, pseudo_labels[mask]])
    train_and_evaluate_classifier(classifier_class, classifier_args, x_combined, y_combined, x_test_pca, y_test)

    # 9. Allena e valuta su tutto il train set originale (x_train_pca, y_train)
    print("\nModello su tutto il dataset supervisionato:")
    train_and_evaluate_classifier(classifier_class, classifier_args, x_train_pca, y_train, x_test_pca, y_test)
    

### **Utilizzare la funzione `main`**

Specifichiamo adesso un set di parametri richiesti dalla funzione main e utilizziamola. Nello specifico la funzione main ha bisogno di:

* `classifier_class`: quale classificatore utilizzare, ad esempio `'MLPClassifier'`, `'LogisticRegression'` o altri visti in precedenza.
* `classifier_args`: un dizionario contenente i parametri del classificatore scelto, ad esempio un `MLPClassifier` necessiterà del parametro `hidden_layer_sizes`. Dipendentemente da quale classificatore scegliete dovrete creare il dizionario.
* `n_components_pca`: numero di componenti di PCA che vogliamo utilizzare. Se specifichiamo un valore compreso in [0, 1] questo verrà considerato come la percentuale di varianza che vogliamo mentenere.
* `labeled_fraction`: percentuale di dati da usare come insieme etichettato. Si consiglia il valore 0.002 corrispondente allo 0.2%, cioè 16 immagini su 8000.
* `n_clusters`: numero di cluster da utilizzare, nel nostro caso vogliamo che ci sia un cluster per ogni classe, quindi 10.

Infine utilizziamo la funzione main.

In [ ]:
# Parametri
CLASSIFIER_CLASS = MLPClassifier  # Modello da usare, ad esempio LogisticRegression o SVC
CLASSIFIER_ARGS = {
    'max_iter': 20,
    'hidden_layer_sizes': (200,200)  # Aumenta il numero di iterazioni per la convergenza
}
N_COMPONENTS_PCA = 0.95  # Mantiene il 95% della varianza spiegata, o un numero fisso es. 50
LABELED_FRACTION = 0.002   # Frazione di dati da usare come insieme etichettato L
N_CLUSTERS = 10          # Fashion-MNIST ha 10 classi

In [ ]:
main(
    CLASSIFIER_CLASS,
    CLASSIFIER_ARGS,
    N_COMPONENTS_PCA,
    LABELED_FRACTION,
    N_CLUSTERS
)